In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline

In [2]:
#Required text pre-processing libraries are imported
import string
import nltk
import re

# download the stopwords and wordnet corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
# import tokenize from nltk library
from nltk import tokenize
# import WordNetLemmatizer from nltk library
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from nltk.probability import FreqDist

#Required data visualisation libraries are imported
import plotly.express as px
import seaborn as sns 
import matplotlib.pyplot as plt


#Required prediction modelling libraries are imported
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import plot_confusion_matrix, confusion_matrix, precision_recall_curve, auc, roc_curve, accuracy_score, recall_score, classification_report, f1_score, precision_score, precision_recall_fscore_support, roc_auc_score, make_scorer
from sklearn.pipeline import Pipeline

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

import xgboost
from xgboost import XGBClassifier


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jillian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jillian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jillian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jillian/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
tweets=pd.read_csv('./data/sample_tweet_dataset.csv')

In [4]:
pd.set_option('display.max_colwidth', None)

In [5]:
tweets

,author_id,created_at,geo_location,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,text
0,25786908,2022-08-10 21:18:03+00:00,,1557476394418028545,en,7,0,2,2,Twitter Web App,Please Join my free only fans account for exclusive content each week. Thank you. 💋❤️\nJillian Jordan Xo\n\nkittycash7 on only fans 💕\n❣️🍒🌸🎵\n\n@ZARA Satin Dress 👗\n@SuicideSquadWB @suicidesquadRS\n@HarleyQuinnAC ❣️❤️‍🩹\n#onlyfans #Legs #Feet #Tootsies 👣\n\nhttps://t.co/R7R34fVoy1 https://t.co/m70FcWzlRb
1,1033953826192216064,2022-08-10 18:32:59+00:00,1d9a5370a355ab0c,1557434855620853766,en,0,0,1,0,Twitter for iPhone,My favorite work pants are too big on me now I’m going to cry I don’t have new work pant money @ZARA send me a new pair for free
2,462883774,2022-08-10 15:24:32+00:00,,1557387428574498816,en,0,0,0,0,Twitter for iPhone,Keep making your clothes in Turkey @benetton @ZARA @skims etc https://t.co/zQKlwdj2q0
3,412500781,2022-08-10 06:33:12+00:00,,1557253713089150981,en,3,0,1,0,Twitter for Android,lol @ZARA it's the same dress https://t.co/9bwmNhPCRL
4,196630146,2022-08-09 17:24:47+00:00,,1557055304663179267,en,0,0,1,0,Twitter for iPhone,@ZARA help me out bought this khaki shirt and the same one in like a cream colour after one wash the colour has came proper out. Bad batch ? Do we know ? https://t.co/hCYXe4JWV6
5,2298364800,2022-08-09 12:14:31+00:00,29732b67b6d40135,1556977222602440706,en,0,0,0,0,Instagram,Jus Kewl 🐼\n#shilpajoshi \n\n#shirt #whiteshirt @zara \n#denimjeans @zara \n#bag @dior \n#glasses @rayban \n#shoes #vansonshoes \n\nMahadev 💫 @ The Sky Patio Faridabad https://t.co/Cv3fDXN416
6,4510623076,2022-08-09 11:11:00+00:00,,1556961236687982594,en,0,0,0,0,Twitter for iPhone,@TheSocialCTV @cinqaseptnyc @MaisonValentino @ted_baker @ZARA @ALDO_Shoes Loved the green detailing on Jess’s dress.
7,1459121136168579073,2022-08-09 08:31:14+00:00,,1556921030018482176,en,0,0,0,0,Twitter for Android,"@ZARA just wondered if you had any nice fairy dresses in? Do you post to Darlington?\n\nHahaha win for the little people 1, win for the for greedy billionaires 0. BOOOOM x"
8,1156502233644785665,2022-08-09 06:25:07+00:00,,1556889290369548290,en,19,0,2,1,Twitter Web App,Harry for @ZARA 🙌🏻 Awesome inclusion-having a seamstress on set altering Harry’s clothes so he could ‘be himself with ease’ and feel confident 💙Thank you @ZARA 😭 #InclusionMatters #HisHeartSettled @ZebedeeMan @luckyfinproject @ReachCharity https://t.co/nqBfyiYgJV
9,1245158946,2022-08-08 20:00:00+00:00,,1556731976731840518,en,13,0,2,1,Twitter Web App,New week. Let's get it!\n\nMel \nDress: @cinqaseptnyc\n\nLainey \nDress: Moon River\nShoes: @MaisonValentino\n\nCynthia \nDress: @ted_baker\nShoes: @ZARA\n\nJess\nDress: MARELLA\nShoes: @ALDO_Shoes\n\n#WhatWeWore #OOTD https://t.co/yPo1ZD5Cd0


In [6]:
#need general preprocessing function to clean my tweets--lowercase, remove punctuation, remove emojis, etc.

In [7]:
#installing tweet-preprocessor
!pip install tweet-preprocessor

In [8]:
#adding the cleaned (After removal of URLs, Mentions) tweets to a new column as a new feature ‘clean_text’

In [9]:
import preprocessor as p
#forming a separate feature for cleaned tweets
for i,v in enumerate(tweets['text']):
    tweets.loc[i,'text']  = p.clean(v)

In [10]:
pd.set_option('display.max_colwidth', None)

In [11]:
tweets.head()

,author_id,created_at,geo_location,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,text
0,25786908,2022-08-10 21:18:03+00:00,,1557476394418028545,en,7,0,2,2,Twitter Web App,Please Join my free only fans account for exclusive content each week. Thank you. Jillian Jordan Xokittycash7 on only fans Satin Dress
1,1033953826192216064,2022-08-10 18:32:59+00:00,1d9a5370a355ab0c,1557434855620853766,en,0,0,1,0,Twitter for iPhone,My favorite work pants are too big on me now Im going to cry I dont have new work pant money send me a new pair for free
2,462883774,2022-08-10 15:24:32+00:00,,1557387428574498816,en,0,0,0,0,Twitter for iPhone,Keep making your clothes in Turkey etc
3,412500781,2022-08-10 06:33:12+00:00,,1557253713089150981,en,3,0,1,0,Twitter for Android,lol it's the same dress
4,196630146,2022-08-09 17:24:47+00:00,,1557055304663179267,en,0,0,1,0,Twitter for iPhone,help me out bought this khaki shirt and the same one in like a cream colour after one wash the colour has came proper out. Bad batch ? Do we know ?


In [12]:
#important libraries for preprocessing using NLTK
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package wordnet to /Users/jillian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jillian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
#Remove Digits and lower the text (makes it easy to deal with)
tweets['text_clean'] = tweets['text'].astype(str).str.replace('\d+', '')
tweets['text_clean'] = tweets['text_clean'].str.lower()

In [14]:
tweets.head()

,author_id,created_at,geo_location,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,text,text_clean
0,25786908,2022-08-10 21:18:03+00:00,,1557476394418028545,en,7,0,2,2,Twitter Web App,Please Join my free only fans account for exclusive content each week. Thank you. Jillian Jordan Xokittycash7 on only fans Satin Dress,please join my free only fans account for exclusive content each week. thank you. jillian jordan xokittycash on only fans satin dress
1,1033953826192216064,2022-08-10 18:32:59+00:00,1d9a5370a355ab0c,1557434855620853766,en,0,0,1,0,Twitter for iPhone,My favorite work pants are too big on me now Im going to cry I dont have new work pant money send me a new pair for free,my favorite work pants are too big on me now im going to cry i dont have new work pant money send me a new pair for free
2,462883774,2022-08-10 15:24:32+00:00,,1557387428574498816,en,0,0,0,0,Twitter for iPhone,Keep making your clothes in Turkey etc,keep making your clothes in turkey etc
3,412500781,2022-08-10 06:33:12+00:00,,1557253713089150981,en,3,0,1,0,Twitter for Android,lol it's the same dress,lol it's the same dress
4,196630146,2022-08-09 17:24:47+00:00,,1557055304663179267,en,0,0,1,0,Twitter for iPhone,help me out bought this khaki shirt and the same one in like a cream colour after one wash the colour has came proper out. Bad batch ? Do we know ?,help me out bought this khaki shirt and the same one in like a cream colour after one wash the colour has came proper out. bad batch ? do we know ?


In [15]:
#remove punctuation
def punctuation_removal(punc):
    rem_punc = [i for i in punc if i not in string.punctuation]
    after_punc = ''.join(rem_punc)
    return after_punc

In [16]:
tweets['text_clean'] = tweets.text_clean.apply(punctuation_removal)

In [17]:
tweets.head()

,author_id,created_at,geo_location,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,text,text_clean
0,25786908,2022-08-10 21:18:03+00:00,,1557476394418028545,en,7,0,2,2,Twitter Web App,Please Join my free only fans account for exclusive content each week. Thank you. Jillian Jordan Xokittycash7 on only fans Satin Dress,please join my free only fans account for exclusive content each week thank you jillian jordan xokittycash on only fans satin dress
1,1033953826192216064,2022-08-10 18:32:59+00:00,1d9a5370a355ab0c,1557434855620853766,en,0,0,1,0,Twitter for iPhone,My favorite work pants are too big on me now Im going to cry I dont have new work pant money send me a new pair for free,my favorite work pants are too big on me now im going to cry i dont have new work pant money send me a new pair for free
2,462883774,2022-08-10 15:24:32+00:00,,1557387428574498816,en,0,0,0,0,Twitter for iPhone,Keep making your clothes in Turkey etc,keep making your clothes in turkey etc
3,412500781,2022-08-10 06:33:12+00:00,,1557253713089150981,en,3,0,1,0,Twitter for Android,lol it's the same dress,lol its the same dress
4,196630146,2022-08-09 17:24:47+00:00,,1557055304663179267,en,0,0,1,0,Twitter for iPhone,help me out bought this khaki shirt and the same one in like a cream colour after one wash the colour has came proper out. Bad batch ? Do we know ?,help me out bought this khaki shirt and the same one in like a cream colour after one wash the colour has came proper out bad batch do we know


In [18]:
#utilizing same lemmatize/tokenize code from initial data cleaning

In [19]:
# Instantiate a word net lemmatizer object and create helper function to apply it to the dataframe.
lemmatizer = WordNetLemmatizer()

def tokenize_lem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as its own token
    tokens = [word.lower() for sent in tokenize.sent_tokenize(text) for word in tokenize.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return [lemmatizer.lemmatize(token) for token in filtered_tokens]

In [20]:
tweets['text_clean'] = tweets.text_clean.apply(tokenize_lem)

In [21]:
tweets.head()

,author_id,created_at,geo_location,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,text,text_clean
0,25786908,2022-08-10 21:18:03+00:00,,1557476394418028545,en,7,0,2,2,Twitter Web App,Please Join my free only fans account for exclusive content each week. Thank you. Jillian Jordan Xokittycash7 on only fans Satin Dress,"[please, join, my, free, only, fan, account, for, exclusive, content, each, week, thank, you, jillian, jordan, xokittycash, on, only, fan, satin, dress]"
1,1033953826192216064,2022-08-10 18:32:59+00:00,1d9a5370a355ab0c,1557434855620853766,en,0,0,1,0,Twitter for iPhone,My favorite work pants are too big on me now Im going to cry I dont have new work pant money send me a new pair for free,"[my, favorite, work, pant, are, too, big, on, me, now, im, going, to, cry, i, dont, have, new, work, pant, money, send, me, a, new, pair, for, free]"
2,462883774,2022-08-10 15:24:32+00:00,,1557387428574498816,en,0,0,0,0,Twitter for iPhone,Keep making your clothes in Turkey etc,"[keep, making, your, clothes, in, turkey, etc]"
3,412500781,2022-08-10 06:33:12+00:00,,1557253713089150981,en,3,0,1,0,Twitter for Android,lol it's the same dress,"[lol, it, the, same, dress]"
4,196630146,2022-08-09 17:24:47+00:00,,1557055304663179267,en,0,0,1,0,Twitter for iPhone,help me out bought this khaki shirt and the same one in like a cream colour after one wash the colour has came proper out. Bad batch ? Do we know ?,"[help, me, out, bought, this, khaki, shirt, and, the, same, one, in, like, a, cream, colour, after, one, wash, the, colour, ha, came, proper, out, bad, batch, do, we, know]"


In [22]:
#removing stop words

In [23]:
sw = stopwords.words('english')

# Helper function to remove stopwords
def remove_stopwords(token_list):
    """
    Given a list of tokens, return a list where the tokens
    that are also present in sw have been removed
    """
    stopwords_r = [token for token in token_list if token not in sw]
    return stopwords_r

In [24]:
# Apply the removal function to df.clean
tweets.text_clean=tweets.text_clean.apply(remove_stopwords)

# create new column from df_clean that converts list of tokens to a string.
tweets['string']=tweets.text_clean.str.join(" ")

In [25]:
tweets.head()

,author_id,created_at,geo_location,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,text,text_clean,string
0,25786908,2022-08-10 21:18:03+00:00,,1557476394418028545,en,7,0,2,2,Twitter Web App,Please Join my free only fans account for exclusive content each week. Thank you. Jillian Jordan Xokittycash7 on only fans Satin Dress,"[please, join, free, fan, account, exclusive, content, week, thank, jillian, jordan, xokittycash, fan, satin, dress]",please join free fan account exclusive content week thank jillian jordan xokittycash fan satin dress
1,1033953826192216064,2022-08-10 18:32:59+00:00,1d9a5370a355ab0c,1557434855620853766,en,0,0,1,0,Twitter for iPhone,My favorite work pants are too big on me now Im going to cry I dont have new work pant money send me a new pair for free,"[favorite, work, pant, big, im, going, cry, dont, new, work, pant, money, send, new, pair, free]",favorite work pant big im going cry dont new work pant money send new pair free
2,462883774,2022-08-10 15:24:32+00:00,,1557387428574498816,en,0,0,0,0,Twitter for iPhone,Keep making your clothes in Turkey etc,"[keep, making, clothes, turkey, etc]",keep making clothes turkey etc
3,412500781,2022-08-10 06:33:12+00:00,,1557253713089150981,en,3,0,1,0,Twitter for Android,lol it's the same dress,"[lol, dress]",lol dress
4,196630146,2022-08-09 17:24:47+00:00,,1557055304663179267,en,0,0,1,0,Twitter for iPhone,help me out bought this khaki shirt and the same one in like a cream colour after one wash the colour has came proper out. Bad batch ? Do we know ?,"[help, bought, khaki, shirt, one, like, cream, colour, one, wash, colour, ha, came, proper, bad, batch, know]",help bought khaki shirt one like cream colour one wash colour ha came proper bad batch know


In [44]:
final_tweets = tweets['string']

In [26]:
#ready to run my tweets through my model to predict proba!

In [27]:
#loading in dataset to train my final model

In [28]:
df = pd.read_csv('./data/singe_word_data.csv')

In [29]:
df = df[df.String.isna() == False]

In [30]:
# Specify X as the cleaned strings in df and y as the target-Positive_Rating.
X = df['String']
y = df['Positive_Rating']

In [31]:
#Performed the train-test split, using 20% for the hold-out data.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    test_size=0.20)

In [32]:
best_model2 = ImPipeline([
           ('vect', TfidfVectorizer(max_df=.5, max_features=3000, ngram_range=(1,2))),
           ('lr', LogisticRegression(random_state=42, class_weight=None, solver='saga')),
])

In [33]:
best_model2.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(max_df=0.5, max_features=3000,
                                 ngram_range=(1, 2))),
                ('lr', LogisticRegression(random_state=42, solver='saga'))])

In [45]:
best_model2.predict(final_tweets)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1])

In [46]:
best_model2.predict_proba(final_tweets)

array([[0.259767  , 0.740233  ],
       [0.24253594, 0.75746406],
       [0.44359382, 0.55640618],
       [0.25932844, 0.74067156],
       [0.47796397, 0.52203603],
       [0.19156994, 0.80843006],
       [0.0925807 , 0.9074193 ],
       [0.09958275, 0.90041725],
       [0.19473342, 0.80526658],
       [0.21226311, 0.78773689],
       [0.37590642, 0.62409358],
       [0.39605857, 0.60394143],
       [0.3728294 , 0.6271706 ],
       [0.72556442, 0.27443558],
       [0.82220672, 0.17779328],
       [0.70997126, 0.29002874],
       [0.16622488, 0.83377512],
       [0.11042959, 0.88957041],
       [0.1176676 , 0.8823324 ],
       [0.21964609, 0.78035391],
       [0.32794054, 0.67205946],
       [0.14343335, 0.85656665],
       [0.10471693, 0.89528307],
       [0.63560585, 0.36439415],
       [0.40341234, 0.59658766],
       [0.28391252, 0.71608748],
       [0.10262307, 0.89737693],
       [0.73028257, 0.26971743],
       [0.70997126, 0.29002874],
       [0.4861521 , 0.5138479 ],
       [0.

In [47]:
#IT WORKED!

In [48]:
final_tweets

0                                                                    please join free fan account exclusive content week thank jillian jordan xokittycash fan satin dress
1                                                                                         favorite work pant big im going cry dont new work pant money send new pair free
2                                                                                                                                          keep making clothes turkey etc
3                                                                                                                                                               lol dress
4                                                                             help bought khaki shirt one like cream colour one wash colour ha came proper bad batch know
5                                                                                                                                    jus kewl mahadev 

In [34]:
#feed tweets to my prepocessing function that matches the steps I took for my final model

In [35]:
#might have to create more generalized preprocessing with regex for the Tweets to be more generalizable

In [36]:
#in presentation discuss how this is generalizable to Instagram, discussion boards, Reddit, Discord, YouTube reviews etc. etc. 

In [37]:
#use for promo boxes/unboxing/clothing haul

In [38]:
#different price points reflected on different forums

In [39]:
#pipeline contains vectorizer (that was trained with training data)

In [40]:
#run my tweets and do a .predict (probably the ones I can predict with my own eyes)

In [41]:
#.predictproba for BEST retweetable (depending on model)

In [42]:
#pickling is deploying in app

In [43]:
#potential limitations: disproportion of positive reviews but people may disproportionately go online to complain (disproportionate negative reviews)